In [3]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Load the dataset
file_path = "/raid/home/somayeh.shami/project/somayeh_workspace/Fertility_project/Final_Cleaned_Fertility_Dataset.xlsx"
data = pd.read_excel(file_path)

# Binary labels: 0 for "Fertile", 1 for "Infertile"
data['binary_label'] = (data['fertility_classification'] == 'Infertile').astype(int)

# Drop irrelevant columns and separate features and labels
X = data.drop(columns=['patient_id', 'fertility_classification', 'infertility_likelihood_(%)', 'binary_label'])
y = data['binary_label']

# Separate categorical and numerical features
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns

# One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
categorical_encoded = encoder.fit_transform(X[categorical_cols])

# Normalize numerical features
scaler = StandardScaler()
numerical_scaled = scaler.fit_transform(X[numerical_cols])

# Combine numerical and categorical features
X_processed = np.hstack([numerical_scaled, categorical_encoded])

# Train-test-validation split
X_train, X_temp, y_train, y_temp = train_test_split(X_processed, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Output shapes and samples to verify
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Sample of validation features:\n{X_val[:5]}")
print(f"Sample of validation labels:\n{y_val[:5].values}")

Training data shape: (697, 90)
Validation data shape: (150, 90)
Test data shape: (150, 90)
Sample of validation features:
[[ 1.33850723  1.24123785  0.          1.          0.          0.
   1.          1.          0.          0.          0.          1.
   0.          0.          0.          1.          0.          0.
   1.          0.          0.          1.          0.          0.
   1.          0.          0.          1.          0.          0.
   0.          0.          1.          0.          1.          0.
   0.          1.          0.          0.          1.          0.
   0.          1.          0.          0.          0.          0.
   1.          0.          0.          1.          1.          0.
   0.          0.          0.          0.          1.          0.
   0.          0.          1.          0.          0.          0.
   1.          0.          0.          1.          0.          0.
   1.          0.          1.          0.          0.          0.
   0.          1.   

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the DNN model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer with updated features
    Dropout(0.5),  # Dropout for regularization
    Dense(64, activation='relu'),  # Hidden layer
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC()])

# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32)

# Evaluate the model on the test set
test_loss, test_accuracy, test_auc = model.evaluate(X_test, y_test)

# Print test performance
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test AUC: {test_auc}")

2025-01-09 09:52:02.537291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736416322.555515 1453112 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736416322.561047 1453112 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 09:52:02.579528: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20


/raid/home/somayeh.shami/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1736416350.774315 1453112 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6084 - auc: 0.5264 - loss: 0.6447 - val_accuracy: 0.8000 - val_auc: 0.5376 - val_loss: 0.5110
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7921 - auc: 0.5747 - loss: 0.5291 - val_accuracy: 0.8000 - val_auc: 0.5286 - val_loss: 0.5093
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7685 - auc: 0.4913 - loss: 0.5611 - val_accuracy: 0.8000 - val_auc: 0.5285 - val_loss: 0.5045
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7745 - auc: 0.5201 - loss: 0.5434 - val_accuracy: 0.8000 - val_auc: 0.5263 - val_loss: 0.5079
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7819 - auc: 0.5611 - loss: 0.5282 - val_accuracy: 0.8000 - val_auc: 0.5283 - val_loss: 0.5047
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7715 - auc: 0.5419 - loss: 0.5521 - val_accuracy: 0.8000 - val_auc: 0.5160 - val_loss: 0.5086
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accu